In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import xml.etree.ElementTree as ET
import warnings
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')

#tree=ET.parse('/content/drive/MyDrive/pubmed20n1001.xml')
documents=pd.read_csv("/content/drive/MyDrive/test.csv",index_col=0)


In [16]:
documents.head()

,No.,Title,Abstract,Chemicals,Meshes,Keywords,doi
0,3-1,Evidence for the presence of an antitumor fact...,('We have previously reported finding a factor...,"['Alpha-Globulins', 'Endotoxins']","['Alpha-Globulins', 'Animals', 'Cattle', 'Cyto...",[],['10.1016/s0304-3835(79)80039-7']
1,13-1,Tumour-necrosis factor from the rabbit. III. R...,('Tumour-necrosis factor (TNF) is growth-inhib...,"['Endotoxins', 'Glycoproteins', 'Growth Inhibi...","['Animals', 'Cell Line', 'Endotoxins', 'Glycop...",[],['10.1038/bjc.1979.218']
2,15-1,Effects of murine tumor necrosis factor on het...,('A partially purified glycoprotein fraction (...,['Glycoproteins'],"['Adenocarcinoma', 'Animals', 'Cells, Cultured...",[],['10.1159/000162922']
3,23-1,Tumour-necrosis factor from the rabbit. I. Mod...,('Sera from rabbits injected with BCG and then...,"['Azides', 'Dinitrophenols', 'Glycoproteins', ...","['Animals', 'Azides', 'Cell Division', 'Cell L...",[],['10.1038/bjc.1978.202']
4,23-2,Tumour-necrosis factor from the rabbit. II. Pr...,('Mononuclear cells from normal rabbit blood w...,['Glycoproteins'],"['Animals', 'Cell Adhesion', 'Cell Line', 'Cyt...",[],['10.1038/bjc.1978.203']


In [4]:
documents.shape

(130584, 7)

In [ ]:
root=tree.getroot()

Abstract=root.findall('./PubmedArticle/MedlineCitation/Article/Abstract')

abstractList=[]
for abstract in Abstract:
  abstractText=abstract.find('AbstractText').text
  if type(abstractText)==str:
    abstractList.append(abstractText.lower())

In [3]:
#Title과 Abstract 추출
title_abstarct=[]
for i in range(documents.shape[0]):
  title_abstarct.append(str(documents.iloc[i]['Title'])+str(documents.iloc[i]['Abstract']).lower())

In [18]:
len(title_abstarct)

130584

In [4]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

word_List=[]
for sentence in title_abstarct:
  words=word_tokenize(sentence)
  word_List.append(words)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
def is_digit(str):
  try:
    tmp = float(str)
    return True
  except ValueError:
    return False

In [6]:
#불용어 제거,특수문자 제거,숫자 제거
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words=set(stopwords.words('english'))
remove_set=['.',',',';','(',')',':','\'','"','?',':','``','-','\'\'','%']
result=[]
for sentence in word_List:
  remove_stopword=[]
  sentence=[i for i in sentence if i not in remove_set]
  for word in sentence:
    if word not in stop_words:
      if not is_digit(word):
        remove_stopword.append(word)
  result.append(remove_stopword)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
from gensim.models.word2vec import Word2Vec
model=Word2Vec(result,sg=1)

In [8]:
from sklearn.metrics.pairwise import euclidean_distances
def get_euclidean_distance(input_word):
  similarity_words=model.wv.most_similar(input_word)
  distances=[]
  for similarity in similarity_words:
    smilarity_word=similarity[0]
    distance=euclidean_distances(model['tnf-α'].reshape(1,-1),model[smilarity_word].reshape(1,-1))
    distances.append(distance)
  return distances

In [9]:
import pandas as pd
import numpy as np

euclidean_distance_list=get_euclidean_distance('tnf-α')
print('TNF-α과 가장 유사한 단어와 유사도')
df=pd.DataFrame(model.wv.most_similar('tnf-α'),columns=['word','cosine similarity'])
df['euclidean_distance']=np.array(get_euclidean_distance('tnf-α')).reshape(-1,1)
df

TNF-α과 가장 유사한 단어와 유사도


,word,cosine similarity,euclidean_distance
0,tnfα,0.879759,1.554270
1,tnf-alpha,0.826700,1.817633
2,il-1β,0.815510,1.979569
3,il-6,0.789414,2.029509
4,tnf-a,0.743355,2.240841
5,il-β,0.692353,2.407773
6,tnf,0.691587,2.497618
7,tnfalpha,0.691236,2.399844
8,ifn-γ,0.685577,2.650009
9,also,0.673916,2.393908


In [10]:
vocab=model.wv.vocab.keys()
vocab=[x for x in vocab]
vocab

['Evidence',
 'presence',
 'antitumor',
 'factor',
 'serum',
 'normal',
 'animals',
 "'we",
 'previously',
 'reported',
 'finding',
 'activity',
 'tnf',
 'tumor',
 'necrosis',
 'extracts',
 'mice',
 'possibility',
 'exists',
 'blood',
 'species',
 'explored',
 'horse',
 'mouse',
 'dog',
 'human',
 'sheep',
 'calf',
 'rat',
 'shark',
 'serums',
 'fractionated',
 'nh4',
 '2so4',
 'filtered',
 's-200',
 'sephacryl',
 'gel',
 'proteins',
 'molecular',
 'weight',
 '90,000',
 'pooled',
 'concentrated',
 'dialyzed',
 'determined',
 'l-cell',
 'assay',
 'vitro',
 'meth',
 'vivo',
 'found',
 'fractions',
 'agarose',
 'electrophoresis',
 'indicated',
 'principle',
 'components',
 'alpha',
 '1-alpha',
 'globulins',
 'preparative',
 'page',
 'migrated',
 'slowly',
 'made',
 'least',
 'faster',
 'moving',
 'monomeric',
 'protein',
 'Tumour-necrosis',
 'rabbit',
 'III',
 'Relationship',
 'interferons',
 "'tumour-necrosis",
 'growth-inhibitory',
 'cytotoxic',
 'certain',
 'tumour',
 'cell',
 'lines',

In [11]:
#TF-IDF 구현
from math import log
N=len(result)

def tf(t,d):
  return d.count(t)

def idf(t):
  df=0
  for doc in result:
    df+=t in doc
  return log(N/(df+1))

def tfidf(t,d):
  return tf(t,d)*idf(t)

In [ ]:
tf_list=[]
for i in range(N):
  tf_list.append([])
  d=result[i]
  for j in range(len(vocab)):
    t=vocab[j]
    tf_list[-1].append(tfidf(t,d))

tf_ = pd.DataFrame(result, columns = vocab)
tf_